In [77]:
# DEPENDENCIES

import pysam
import pandas as pd
import HTSeq
import GeneImage
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
# from tqdm.notebook import tqdm, trange
from tqdm import tqdm as tqdm
import cProfile
import plotly.io as pio
pio.renderers.default = 'notebook_connected'

# GLOBAL VARS

FC30_DMGOTH_BAMFILE = "/data2/eric/TE_LR_RNAseq/data/dmgoth101_genome_alignments/bam/FC30.against_dmgoth.bam"
FC30_DMGOTH_MAX_AS_BAMFILE = "/data2/eric/TE_LR_RNAseq/data/dmgoth101_genome_alignments/bam/FC30.against_dmgoth.filtered_max_AS.bam"
FC30_DMGOTH_MAX_AS_PRIMARY_ONLY_BAMFILE ="/data2/eric/TE_LR_RNAseq/data/dmgoth101_genome_alignments/bam/FC30.against_dmgoth.filtered_max_AS.primary_only.bam"

FC29_DMGOTH_BAMFILE = "/data2/eric/TE_LR_RNAseq/data/dmgoth101_genome_alignments/bam/FC29.against_dmgoth.bam"
FC29_DMGOTH_MAX_AS_BAMFILE = "/data2/eric/TE_LR_RNAseq/data/dmgoth101_genome_alignments/bam/FC29.against_dmgoth.filtered_max_AS.bam"
FC29_DMGOTH_MAX_AS_PRIMARY_ONLY_BAMFILE ="/data2/eric/TE_LR_RNAseq/data/dmgoth101_genome_alignments/bam/FC29.against_dmgoth.filtered_max_AS.primary_only.bam"

FC30_DMGOTH_SUBSET_BAMFILE = "/data2/eric/TE_LR_RNAseq/data/dmgoth101_genome_alignments/bam/FC30.subset.against_dmgoth.bam"
FC30_DMGOTH_SUBSET_MAX_AS_BAMFILE = "/data2/eric/TE_LR_RNAseq/data/dmgoth101_genome_alignments/bam/FC30.subset.against_dmgoth.filtered.bam"

REPEATMASKER_TE_ANNOTATIONS = "/data2/eric/TE_LR_RNAseq/data/dmgoth101_genome_alignments/annotations/dmgoth101.onecode.v2.gtf"
DMGOTH_GENE_ANNOTATIONS = "/data2/eric/TE_LR_RNAseq/data/dmgoth101_genome_alignments/annotations/Dm_Goth_10-1.dmel6.23LiftOff.sorted.gff"

DFAM_TE_ANNOTATION = "/data2/eric/TE_LR_RNAseq/data/families.flanked_LTR.hierarchy.fa"

INSERTION_TABLE = "/data2/eric/TE_LR_RNAseq/data/dmgoth101_genome_alignments/insertion_table.tsv"

In [78]:
def get_TE_hierarchy(consensus_fasta):
    subclass_list = []
    superfamily_list = []
    family_list = []
    with open(consensus_fasta, 'r') as consensus:
        for line in consensus:
            if line.startswith(">"):
                family = line.split("#")[0][1:]
                subclass, superfamily= line.strip().split("#")[1].split("/")
                subclass_list.append(subclass)
                superfamily_list.append(superfamily)
                family_list.append(family)
    hierarchy_df = pd.DataFrame(list(zip(subclass_list, superfamily_list, family_list)), columns =['Subclass', 'Superfamily', 'Family'])
    return hierarchy_df

def manually_fix_hierarchy(hierarchy_df):
    gypsy6A_row = {"Subclass":"LTR", "Superfamily":"Gypsy", "Family":"Gypsy6A"}
    gypsy12A_row = {"Subclass":"LTR", "Superfamily":"Gypsy", "Family":"Gypsy12A"}
    dm412B_row = {"Subclass":"LTR", "Superfamily":"Gypsy", "Family":"DM412B"}
    hmsbeagle_row = {"Subclass":"LTR", "Superfamily":"Gypsy", "Family":"HMSBEAGLE"}
    dmtom1_row = {"Subclass":"LTR", "Superfamily":"Gypsy", "Family":"DMTOM1"}
    mudr1_row = {"Subclass":"Unknown", "Superfamily":"Unknown", "Family":"MuDR-1_DEl"}
    stalker3_row = {"Subclass":"LTR", "Superfamily":"Gypsy", "Family":"Stalker3"}
    P_row = {"Subclass":"TIR", "Superfamily":"P", "Family":"P-1_DY"}
    rehavkus_row = {"Subclass":"Unknown", "Superfamily":"Unknown", "Family":"Rehavkus-1_DY"}
    new_row_list = [gypsy6A_row, gypsy12A_row, dm412B_row, hmsbeagle_row, dmtom1_row, mudr1_row, stalker3_row, P_row,rehavkus_row]
    for new_row in new_row_list :
        hierarchy_df = hierarchy_df.append(new_row, ignore_index=True)
    return hierarchy_df

In [79]:
HIERARCHY_DF = get_TE_hierarchy(DFAM_TE_ANNOTATION)
HIERARCHY_DF = manually_fix_hierarchy(HIERARCHY_DF)



In [25]:
def get_TE_hierarchy(consensus_fasta):
    subclass_list = []
    superfamily_list = []
    family_list = []
    with open(consensus_fasta, 'r') as consensus:
        for line in consensus:
            if line.startswith(">"):
                family = line.split("#")[0][1:]
                subclass, superfamily= line.strip().split("#")[1].split("/")
                subclass_list.append(subclass)
                superfamily_list.append(superfamily)
                family_list.append(family)
    hierarchy_df = pd.DataFrame(list(zip(subclass_list, superfamily_list, family_list)), columns =['Subclass', 'Superfamily', 'Family'])
    return hierarchy_df

def manually_fix_hierarchy(hierarchy_df):
    gypsy6A_row = {"Subclass":"LTR", "Superfamily":"Gypsy", "Family":"Gypsy6A"}
    gypsy12A_row = {"Subclass":"LTR", "Superfamily":"Gypsy", "Family":"Gypsy12A"}
    dm412B_row = {"Subclass":"LTR", "Superfamily":"Gypsy", "Family":"DM412B"}
    hmsbeagle_row = {"Subclass":"LTR", "Superfamily":"Gypsy", "Family":"HMSBEAGLE"}
    dmtom1_row = {"Subclass":"LTR", "Superfamily":"Gypsy", "Family":"DMTOM1"}
    mudr1_row = {"Subclass":"Unknown", "Superfamily":"Unknown", "Family":"MuDR-1_DEl"}
    stalker3_row = {"Subclass":"LTR", "Superfamily":"Gypsy", "Family":"Stalker3"}
    P_row = {"Subclass":"TIR", "Superfamily":"P", "Family":"P-1_DY"}
    rehavkus_row = {"Subclass":"Unknown", "Superfamily":"Unknown", "Family":"Rehavkus-1_DY"}
    new_row_list = [gypsy6A_row, gypsy12A_row, dm412B_row, hmsbeagle_row, dmtom1_row, mudr1_row, stalker3_row, P_row,rehavkus_row]
    for new_row in new_row_list :
        hierarchy_df = hierarchy_df.append(new_row, ignore_index=True)
    return hierarchy_df

In [80]:
### REBOOT
def gene_id_to_family_name(gene_id):
    family_name = gene_id
    suffix_list = ["_I", "-I_DM", "_I_DM", "_LTR", "-LTR_DM", "_LTR_DM"]
    for suffix in suffix_list:
        if gene_id[-len(suffix):] == suffix:
            family_name = gene_id[:-len(suffix)]
            break
    return family_name

class TE_feature:
    def __init__(self, chrom, start, end, gene_id, insertion_id):
        self.chrom = chrom
        self.start = start
        self.end = end
        # self.gene_id = gene_id
        self.insertion_id = insertion_id
        self.count = 0
        self.family = gene_id_to_family_name(gene_id)
        self.counted_reads = {}
    
    def __len__(self):
        return self.end - self.start
    
    def __repr__(self):
        return self.insertion_id
    
    def is_valid_feature(self):
        return len(self) > 150
    
def parse_TE_annotation_line(line):
    sline = line.strip().split("\t")
    chrom = sline[0]
    start = int(sline[3])
    end = int(sline[4])
    gene_id = sline[-1].split(";")[0].split('"')[1]
    insertion_id = sline[-1].strip().split('transcript_id "')[-1][:-2]

    return TE_feature(chrom, start, end, gene_id, insertion_id)

def parse_TE_annotation_file(annotations):
    dict_of_valid_TE = {}
    with open(annotations, 'r') as annot:
        for line in annot :
            new_feature = parse_TE_annotation_line(line)
            if new_feature.is_valid_feature:
                if new_feature.chrom not in dict_of_valid_TE:
                    dict_of_valid_TE[new_feature.chrom] = [new_feature]
                else :
                    dict_of_valid_TE[new_feature.chrom].append(new_feature)
    for chrom in dict_of_valid_TE :
        dict_of_valid_TE[chrom].sort(key=lambda x: x.start, reverse=False)
    return dict_of_valid_TE

def get_reads_mapped_on_TE_feature(alignment_file, TE_annotation):
    reads_mapped_on_TE = []
    dict_of_valid_TE = parse_TE_annotation_file(TE_annotation)
    with pysam.AlignmentFile(alignment_file, 'rb') as bam:
        for chrom, insertion_list in tqdm(dict_of_valid_TE.items(), desc="Chromosome", position=0, leave=True):
            for insertion in tqdm(insertion_list, desc =chrom, position=0, leave=True):
                reads_mapped_on_TE += list(bam.fetch(contig=insertion.chrom, start=insertion.start, stop=insertion.end))
    return set(reads_mapped_on_TE)

In [81]:
class Gene:
    def __init__(self, chrom, start, end, gene_id):
        self.chrom = chrom
        self.start = start
        self.end = end
        self.gene_id = gene_id
    
    def __len__(self):
        return self.end - self.start
    
    def __repr__(self):
        return self.gene_id

def parse_gene_annotation_line(line):
    sline = line.strip().split("\t")
    chrom = sline[0]
    start = int(sline[3])
    end = int(sline[4])
    gene_id = sline[-1].split(";")[0].split('"')[1]
    return Gene(chrom, start, end, gene_id)


def is_overlapped(start1, end1, start2, end2):
    return end1 >= start2 and end2 >= start1

def get_overlapped_TE(read, dict_of_valid_TE):
    list_of_overlapped_TE =[]
    chrom = read.reference_name
    for insertion in dict_of_valid_TE[chrom]:
        if read.reference_end < insertion.start:
            continue
        if read.reference_end >= insertion.start and insertion.end >= read.reference_start :
            list_of_overlapped_TE.append(insertion)
    return list_of_overlapped_TE

def get_dict_of_overlapped_gene(dict_of_valid_TE, gene_annotation):
    dict_of_overlapped_gene = {}
    with open(gene_annotation, 'r') as gene_annot:
        for line in gene_annot:
            sline = line.strip().split('\t')
            if sline[2] == "gene" :
                chrom = sline[0]
                gene_start = int(sline[3])
                gene_stop = int(sline[4])
                if chrom in dict_of_valid_TE:
                    for insertion in dict_of_valid_TE[chrom]:
                        if is_overlapped(insertion.start, insertion.end, gene_start, gene_stop):
                            new_gene = parse_gene_annotation_line(line)
                            if chrom in dict_of_overlapped_gene:
                                dict_of_overlapped_gene[chrom].append(new_gene)
                            else:
                                dict_of_overlapped_gene[chrom] = [new_gene]
    return dict_of_overlapped_gene   
    
def get_overlapped_gene(read, dict_of_overlapped_gene):
    list_of_overlapped_gene =[]
    for gene in dict_of_overlapped_gene[read.reference_name]:
        if read.reference_end < gene.start:
            continue
        if is_overlapped(read.reference_start, read.reference_end, gene.start, gene.end):
            list_of_overlapped_gene.append(gene)
    return list_of_overlapped_gene

def choose_optimal_feature(read, list_of_overlapped_feature):
    min_dist = 999999999999
    optimal_feature = None
    for overlapped_feature in list_of_overlapped_feature:
        start_dist = abs(overlapped_feature.start - read.reference_start)
        end_dist = abs(overlapped_feature.end - read.reference_end)
        total_dist = start_dist + end_dist
        if total_dist < min_dist:
            min_dist = total_dist
            optimal_feature = overlapped_feature
    return optimal_feature

def get_subject_coverage(alignment, insertion):
    overlap_start = max(alignment.reference_start, insertion.start)
    overlap_end = min(alignment.reference_end, insertion.end)
    overlap_length = overlap_end - overlap_start
    insertion_length = insertion.end - insertion.start
    subject_coverage = overlap_length / insertion_length
    return subject_coverage

def default_filter(alignment, insertion):
    subject_coverage = get_subject_coverage(alignment, insertion)
    nb_aligned_pairs =  alignment.get_overlap(insertion.start, insertion.end)
    is_ok = (subject_coverage > 0.1 and nb_aligned_pairs > 1)
    return is_ok

def get_exceeding_alignment_length(alignment, insertion):
    overflow_length = 0
    if alignment.reference_start < insertion.start:
        overflow_length += insertion.start -alignment.reference_start
    if alignment.reference_end > insertion.end:
        overflow_length += alignment.reference_end - insertion.end
    return overflow_length

def filter_out_co_expressed(alignment, insertion):
    exceeding_length = get_exceeding_alignment_length(alignment, insertion)
    alignment_length = alignment.reference_end - alignment.reference_start
    is_ok = (exceeding_length < (0.1 * alignment_length))
    return is_ok

def map_reads_to_TE(reads_mapped_on_TE_feature, bamfile, TE_annotations, gene_annotations):
    insertion_countings = {}
    dict_of_valid_TE = parse_TE_annotation_file(TE_annotations)
    dict_of_overlapped_gene = get_dict_of_overlapped_gene(dict_of_valid_TE, gene_annotations)
    for read in tqdm(reads_mapped_on_TE_feature, position=0, leave=True):
        list_of_overlapped_gene = get_overlapped_gene(read, dict_of_overlapped_gene)
        list_of_overlapped_TE = get_overlapped_TE(read, dict_of_valid_TE)
        best_feature = choose_optimal_feature(read, list_of_overlapped_TE + list_of_overlapped_gene)
        if isinstance(best_feature, TE_feature):
            ################### FILTERS GO HERE ########################
            if default_filter(read, best_feature) and filter_out_co_expressed(read, best_feature):
                best_feature.count += 1
                best_feature.counted_reads[read.query_name] = read
    for insertion_list in dict_of_valid_TE.values():
        for insertion in insertion_list:
            if insertion.family not in insertion_countings:
                insertion_countings[insertion.family] = {insertion.insertion_id : insertion}
            else :
                insertion_countings[insertion.family][insertion.insertion_id] = insertion
    return insertion_countings

In [82]:
dict_of_valid_TE = parse_TE_annotation_file(REPEATMASKER_TE_ANNOTATIONS)
DICT_OF_INSERTION = {}
for TE_list in dict_of_valid_TE.values():
    for feature in TE_list:
        DICT_OF_INSERTION[feature.insertion_id] = feature 

In [83]:
def get_reads_counted_on_feature(feature_id, reads_mapped_on_TE_feature, bamfile, TE_annotations, gene_annotations):
    read_list = []
    dict_of_valid_TE = parse_TE_annotation_file(TE_annotations)
    dict_of_overlapped_gene = get_dict_of_overlapped_gene(dict_of_valid_TE, gene_annotations)
    feature_chrom = None
    for chrom, feature_list in dict_of_valid_TE.items():
        if feature_id in [f.insertion_id for f in feature_list]:
            feature_chrom = chrom
    if feature_chrom == None :
        return 0
    for read in tqdm(reads_mapped_on_TE_feature, position=0, leave=True):
        if read.reference_name == feature_chrom:
            list_of_overlapped_gene = get_overlapped_gene(read, dict_of_overlapped_gene)
            list_of_overlapped_TE = get_overlapped_TE(read, dict_of_valid_TE)
            best_feature = choose_optimal_feature(read, list_of_overlapped_TE + list_of_overlapped_gene)
            if isinstance(best_feature, TE_feature):
                if best_feature.insertion_id == feature_id:
                    if default_filter(read, best_feature) and filter_out_co_expressed(read, best_feature):
                        read_list.append(read)
    return read_list

def from_counting_to_dataframe(insertion_countings):
    subclass_list = []
    superfamily_list = []
    family_list = []
    insertion_list = []
    counting_list = []
    counted_reads = []
    for family, family_insertion_dict in insertion_countings.items() :
        try:
            subclass = HIERARCHY_DF.loc[HIERARCHY_DF['Family'] == family, 'Subclass'].values[0]
            superfamily = HIERARCHY_DF.loc[HIERARCHY_DF['Family'] == family, 'Superfamily'].values[0]
        except KeyError:
            print(family + " hierarchy unknown in the consensus file : tagged as Unknown...")
            subclass = "Unknown"
            superfamily = "Unknown"
        for insertion_id, insertion in family_insertion_dict.items():
            family_list.append(family)
            insertion_list.append(insertion_id)
            counting_list.append(insertion.count)
            superfamily_list.append(superfamily)
            subclass_list.append(subclass)
            counted_reads.append(insertion.counted_reads)
    df = pd.DataFrame(list(zip(subclass_list, superfamily_list, family_list, insertion_list, counting_list, counted_reads)),
               columns =['Subclass', 'Superfamily', "Family", "Insertion", "Counting", "Reads"])
    return df

In [84]:
def get_subject_coverage(alignment, insertion):
    overlap_start = max(alignment.reference_start, insertion.start)
    overlap_end = min(alignment.reference_end, insertion.end)
    overlap_length = overlap_end - overlap_start
    insertion_length = insertion.end - insertion.start
    if insertion_length != 0 :
        subject_coverage = overlap_length / insertion_length
    else :
        subject_coverage = 0
    return subject_coverage

def get_insertion_mean_subject_coverage(insertion_id, counted_reads):
    insertion = DICT_OF_INSERTION[insertion_id]
    mean_subject_coverage = 0
    for read in counted_reads.values():
        mean_subject_coverage += get_subject_coverage(read, insertion)
    if len(counted_reads) == 0 :
        return 0
    else:
        return mean_subject_coverage/len(counted_reads)

def add_mean_subject_coverage_to_df(counting_df):
    counting_df["mean_subcov"] = counting_df.apply(lambda x: get_insertion_mean_subject_coverage(x['Insertion'], x['Reads']), axis=1)
    return counting_df


In [69]:
def count_TE_feature(alignment_file, TE_annotation, gene_annotation):
    reads_mapped_on_TE_feature = get_reads_mapped_on_TE_feature(alignment_file, TE_annotation)
    insertion_countings = map_reads_to_TE(reads_mapped_on_TE_feature, alignment_file, TE_annotation, gene_annotation)
    counting_df = from_counting_to_dataframe(insertion_countings)
    add_mean_subject_coverage_to_df(counting_df)
    return counting_df




# with cProfile.Profile() as pr:
#     FC30_counting_df = count_TE_feature(FC29_DMGOTH_MAX_AS_BAMFILE, REPEATMASKER_TE_ANNOTATIONS, DMGOTH_GENE_ANNOTATIONS)
# stats = pstats.Stats(pr)
# stats.sort_stats(pstats.SortKey.TIME)
# stats.dump_stats(filename="profiling_test.prof")
# pd.set_option('display.max_rows', 100)
# FC30_counting_df.sort_values(by=['Counting'], ascending=False).head(100)

In [32]:
FC30_counting_df = count_TE_feature(FC30_DMGOTH_MAX_AS_BAMFILE, REPEATMASKER_TE_ANNOTATIONS, DMGOTH_GENE_ANNOTATIONS)
FC29_counting_df = count_TE_feature(FC29_DMGOTH_MAX_AS_BAMFILE, REPEATMASKER_TE_ANNOTATIONS, DMGOTH_GENE_ANNOTATIONS)


100%|██████████| 127490/127490 [04:54<00:00, 432.88it/s]


In [39]:
FC29_counting_df.sort_values(by=['Counting'], ascending=False).head(100)

,Subclass,Superfamily,Family,Insertion,Counting,Reads,mean_subcov
197,LINE,I-Jockey,TAHRE,TAHRE_2R_RaGOO_1145909_1151824,267,{'d7dedb28-76e2-4740-aeef-22bd7e936577': d7ded...,0.165054
8254,LINE,I-Jockey,TART_B1,TART_B1_2R_RaGOO_23920095_23923160,247,{'444b015d-6ecf-428f-95cd-30b4a206f715': 444b0...,0.623737
257,LINE,I-Jockey,HETA,HETA_3L_RaGOO_12349_18402,141,{'39c656e4-149f-4961-ba3a-682e177bb78f': 39c65...,0.347078
258,LINE,I-Jockey,HETA,HETA_3L_RaGOO_18403_20679,136,{'d8f190c1-7e57-4ad0-8601-fe70baa9c42f': d8f19...,0.357868
241,LINE,I-Jockey,TART-A,TART-A_X_RaGOO_3_3271,105,{'96d23915-1bb0-45a5-9d38-7ac5f6b3b0e5': 96d23...,0.505942
...,...,...,...,...,...,...,...
12362,LTR,Pao,BEL,BEL_LTR_2R_RaGOO_2068892_2074965,6,{'80b7ea0f-631d-450c-ace6-7df76dacce3a': 80b7e...,0.322026
14150,LTR,Gypsy,Gypsy2,Gypsy2-I_DM_2R_RaGOO_545702_556702,6,{'afc4782b-7c65-464f-b52e-43a8a0e8e17b': afc47...,0.441379
617,LTR,Copia,Copia,Copia_I_3R_RaGOO_1738736_1739720,6,{'6f3a25a9-7f11-4f46-af3e-970908a99d43': 6f3a2...,0.190210
2551,RC,Helitron,DNAREP1_DM,DNAREP1_DM_2R_RaGOO_5274469_5275451,6,{'82acda28-ea03-4a5c-bb59-06b80318e8d0': 82acd...,0.585031


In [57]:
def save_counting_df(counting_df, csv_file):
    counting_df.to_csv(csv_file, sep = '\t', index=False)

save_counting_df(FC29_counting_df, "FC29_counting_df.v2.tsv")
save_counting_df(FC30_counting_df, "FC30_counting_df.v2.tsv")

In [85]:
### Import counting

saved_FC29_counting_df = pd.read_csv("FC29_counting_df.v2.tsv", sep = "\t")
filtered_FC29_counting_df = saved_FC29_counting_df[saved_FC29_counting_df['Counting'] != 0]

saved_FC30_counting_df = pd.read_csv("FC30_counting_df.v2.tsv", sep = "\t")
filtered_FC30_counting_df = saved_FC30_counting_df[saved_FC30_counting_df['Counting'] != 0]

In [87]:
filtered_FC29_counting_df["Family"].unique()

array(['DM297', 'TAHRE', 'TART-A', 'HETA', 'MDG1', 'ZAM', 'QUASIMODO',
       'Copia', 'BLASTOPIA', 'HOBO', 'M4DM', 'IDEFIX', 'DNAREP1_DM',
       'BS2', 'QUASIMODO2', 'I_DM', 'ROO', 'FW2_DM', 'BS', 'TRANSPAC',
       'POGO', 'DM412', 'LINEJ1_DM', 'PROTOP_B', 'NOMAD', 'ACCORD2',
       'MICROPIA', 'FB4_DM', 'BURDOCK', 'G2_DM', 'TART_B1', 'S2_DM',
       'S_DM', 'DM1731', 'G4_DM', 'DOC6_DM', 'DMRT1A', 'DOC', 'MDG3',
       'PROTOP_A', 'BARI1', 'DMRT1B', 'DMCR1A', 'DIVER', 'G5A_DM',
       'BS3_DM', 'DOC4_DM', 'Gypsy6A', 'DOC3_DM', 'PROTOP', 'G_DM',
       'G5_DM', 'Transib5', 'Jockey2', 'Copia2', 'Invader3', 'Invader4',
       'TRANSIB2', 'Invader2', 'DMLTR5', 'DMRT1C', 'MAX', 'Chimpo',
       'TIRANT', 'Copia1', 'Baggins1', 'Mariner2_DM', 'DOC2_DM', 'DM412B',
       'Gypsy6', 'BEL', 'Gypsy5', 'DIVER2', 'FW3_DM', 'BATUMI', 'Gypsy4',
       'Gypsy9', 'Invader6', 'Gypsy12A', 'CIRCE', 'Chouto', 'NINJA',
       'Gypsy8', 'Invader1', 'DM176', 'TABOR', 'Gypsy', 'G3_DM',
       'TRANSIB4', 'Bi

In [5]:
def draw_icicle(df):
    fig = px.icicle(df, path=['Subclass', 'Superfamily', 'Family', 'Insertion'], values='Counting',
                    color='mean_subcov',
                    hover_data=['Counting'],
                    color_continuous_scale='RdBu',
                    color_continuous_midpoint=np.average(df['mean_subcov'],weights=df['Counting'])
                    )
    fig.update_traces(root_color="lightgrey")
    fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
    fig.show()

def draw_sunburst(df):
    fig = px.sunburst(df, path=['Subclass', 'Superfamily', 'Family', 'Insertion'], values='Counting',
                    color='mean_subcov',
                    hover_data=['Counting'],
                    color_continuous_scale='RdBu',
                    color_continuous_midpoint=np.average(df['mean_subcov'],weights=df['Counting'])
                    )
    fig.update_traces(root_color="lightgrey")
    fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
    fig.show()

In [58]:
draw_sunburst(filtered_FC30_counting_df)

In [7]:
draw_sunburst(filtered_FC29_counting_df)

In [55]:
pd.set_option('display.max_rows', 100)
# filtered_FC29_counting_df.sort_values(by=['Counting'], ascending=False).head(100)
# print(saved_FC29_counting_df[saved_FC29_counting_df['Insertion'] == "Gypsy_I_2R_RaGOO_2048940_2052346"])
# print(saved_FC29_counting_df[saved_FC29_counting_df['Insertion'] == "Gypsy6_LTR_2R_RaGOO_2049422_2049472"])

,Family,Min_TE_length,Max_TE_length,Counts,nb_of_insertion,nb_of_expressed_insertion,Most_expressed_insertion
4,HETA,37,8921,558,18,13,HETA_3L_RaGOO_12349_18402
2,TAHRE,52,17305,461,28,11,TAHRE_2R_RaGOO_1145909_1151824
8,Copia,80,8723,280,50,11,Copia_LTR_3L_RaGOO_10022428_10027467
33,TART_B1,28,6179,247,48,1,TART_B1_2R_RaGOO_23920095_23923160
9,BLASTOPIA,61,5029,159,49,10,BLASTOPIA_LTR_2R_RaGOO_2878638_2883652
3,TART-A,33,8596,141,27,6,TART-A_X_RaGOO_3_3271
64,Jockey2,39,3419,104,49,4,Jockey2_X_RaGOO_21275337_21276483
21,TRANSPAC,245,5252,94,14,11,TRANSPAC_LTR_3R_RaGOO_21217775_21223024
70,TRANSIB2,38,4053,81,53,13,TRANSIB2_2R_RaGOO_3421509_3423952
107,Gypsy12,56,11395,76,80,6,Gypsy12_LTR_3R_RaGOO_842982_854376


In [11]:
pd.set_option('display.max_rows', 100)
filtered_FC30_counting_df.sort_values(by=['Counting'], ascending=False).head(100)
print(saved_FC30_counting_df[saved_FC30_counting_df['Insertion'] == "Gypsy6_LTR_2R_RaGOO_2049422_2049472"])

      Subclass Superfamily  Family                            Insertion  \
12296      LTR       Gypsy  Gypsy6  Gypsy6_LTR_2R_RaGOO_2049422_2049472   

       Counting Reads  mean_subcov  
12296         0    {}          0.0  


In [88]:
def get_insertion_length(insertion_name):
	start, end = insertion_name.split('_')[-2:]
	return int(end) - int(start) + 1

def create_summary_table(counting_table):
	subclass_list = []
	superfamily_list = []
	family_list = counting_table["Family"].unique()
	min_TE_length_list = []
	max_TE_length_list = []
	counting_list = []
	nb_expressed_insertion_list = []
	nb_insertion_list = []
	most_expressed_insertion_list = []
	for family in family_list :
		family_df = counting_table[counting_table["Family"] == family]
		subclass_list.append(family_df["Subclass"].values[0])
		superfamily_list.append(family_df["Superfamily"].values[0])
		min_TE_length = 9999999
		max_TE_length = 0
		for insertion_name in family_df["Insertion"]:
			insertion_length = get_insertion_length(insertion_name)
			if insertion_length < min_TE_length :
				min_TE_length = insertion_length
			if insertion_length > max_TE_length :
				max_TE_length = insertion_length
		min_TE_length_list.append(min_TE_length)
		max_TE_length_list.append(max_TE_length)
		nb_insertion_list.append(len(family_df["Insertion"]))
		counting_list.append(family_df["Counting"].sum())
		nb_expressed_insertion_list.append(len(family_df[family_df["Counting"] > 0]))
		most_expressed_insertion_list.append(family_df[family_df["Counting"] == family_df["Counting"].max()]["Insertion"].values[0])
    
	summary_df = pd.DataFrame(list(zip(subclass_list, superfamily_list, family_list, min_TE_length_list, max_TE_length_list, counting_list, nb_insertion_list, nb_expressed_insertion_list, most_expressed_insertion_list)), columns=["Subclass", "Superfamily", "Family", "Min_TE_length","Max_TE_length", "Counts", "nb_of_insertion", "nb_of_expressed_insertion", "Most_expressed_insertion"])
	return summary_df


def merge_female_and_male_df(female_counting, male_counting):
	female_df = create_summary_table(female_counting)
	male_df = create_summary_table(male_counting)
	hierarchy_cols = female_df[["Subclass", "Superfamily", "Family", "nb_of_insertion", "Min_TE_length","Max_TE_length"]]
	hierarchy_df = hierarchy_cols.copy()
	full_summary_df = hierarchy_df.merge(female_df.drop(columns=["Subclass", "Superfamily", "Min_TE_length","Max_TE_length", "nb_of_insertion"]), on='Family').merge(male_df.drop(columns=["Subclass", "Superfamily", "Min_TE_length","Max_TE_length", "nb_of_insertion"]), on='Family')
	full_summary_df.columns = ["Subclass", "Superfamily", "Family", "nb_of_insertion", "Min_TE_length", "Max_TE_length", "Female_Counts", "Female_nb_of_expressed_insertion", "Female_most_expressed_insertion", "Male_Counts", "Male_nb_of_expressed_insertion", "Male_most_expressed_insertion"]

	return full_summary_df

merged_full_summary_df = merge_female_and_male_df(saved_FC30_counting_df, saved_FC29_counting_df)
# print(merged_full_summary_df)
save_counting_df(merged_full_summary_df, "merged_full_summary.v2.tsv")

# def create_table_from_counting(female_counting, male_counting, hierarchy):
#     male_counting = male_counting.drop(columns=["Min_TE_length", "Max_TE_length", "nb_of_insertion"])
#     TE_data_table = hierarchy.merge(female_counting, on='Family').merge(male_counting, on='Family')
#     TE_data_table.columns = ["Subclass", "Superfamily", "Family", "Min_TE_length", "Max_TE_length", "Female_Counts", "Nb_of_insertion", "Female_nb_of_expressed_insertion", "Female_most_expressed_insertion", "Male_Counts", "Male_nb_of_expressed_insertion", "Male_most_expressed_insertion"]
#     TE_data_table = TE_data_table[["Subclass", "Superfamily", "Family", "Nb_of_insertion", "Min_TE_length", "Max_TE_length", "Female_Counts", "Female_nb_of_expressed_insertion", "Female_most_expressed_insertion", "Male_Counts", "Male_nb_of_expressed_insertion", "Male_most_expressed_insertion"]]
#     return TE_data_table

In [ ]:
def from_counting_to_df(counting_dict):
    family_list = []
    min_TE_length_list = []
    max_TE_length_list = []
    counting_list = []
    nb_expressed_insertion_list = []
    nb_insertion_list = []
    most_expressed_insertion_list = []
    for family, insertion_list in counting_dict.items():
        family_list.append(family)
        nb_insertion_list.append(len(insertion_list))
        nb_expressed_insertion = 0
        min_TE_length = 9999999999
        max_TE_length = 0
        family_counts = 0
        most_expressed_insertion = None
        max_count = 0
        for insertion in insertion_list :
            insertion_length = insertion.end - insertion.start
            min_TE_length = min(min_TE_length, insertion_length)
            max_TE_length = max(max_TE_length, insertion_length)
            family_counts += insertion.counts
            max_count = max(max_count,insertion.counts)
            if insertion.counts >= max_count :
                most_expressed_insertion = insertion.id
            if insertion.counts > 0 :
                nb_expressed_insertion += 1
        nb_expressed_insertion_list.append(nb_expressed_insertion)
        min_TE_length_list.append(min_TE_length)
        max_TE_length_list.append(max_TE_length)
        counting_list.append(family_counts)
        most_expressed_insertion_list.append(most_expressed_insertion)
        
    counting_df = pd.DataFrame(list(zip(family_list, min_TE_length_list, max_TE_length_list, counting_list, nb_insertion_list, nb_expressed_insertion_list, most_expressed_insertion_list)), columns=["Family", "Min_TE_length","Max_TE_length", "Counts", "nb_of_insertion", "nb_of_expressed_insertion", "Most_expressed_insertion"])
    return counting_df